# Generalising Influence Functions with Autograd

This notebook is an implementation of the data splitting estimator for the Shannon Entropy in 'Influence Functions for Machine Learning: Nonparametric Estimators for Entropies, Divergences and Mutual Informations' by Kandasamy et al. 2015 https://arxiv.org/abs/1411.4342

The process is the same as theirs except, instead of implementing the analytical form derived in the paper, we use autograd.

#### Acknowledgements
I'd like to thank Nic Ford, Sina Akbari, and Jalal Etesami for their patience in helping me work through this topic.

### Form

This work is concerned with functions of the form $$T(p) = \phi \left( \int \nu(p) d\nu \right)$$

where $T(p)$ is the target functional we wish to estimate, and $p$ is a density.

Notice already that Shannon entropy $-\int p \log p$ can be expressed in the same form. There are a couple of ways to do this, but one way is with $\phi(p) = p$ (i.e. the identity function) and $\nu(p) = p \log p$. 

### Pathwise Derivative

The data we collect enables us to estimate $T(p)$ but not the true population quantity $T(q)$. i.e. we have access to $P$ but not to $Q$. We assume that $T(p)$ is 'close enough' and lies on a path to $T(q)$. This allows us to define the pathwise or 'Gateaux' derivative as:

$$
T'(H; P) = \left. \frac{\partial T(P+tH)}{\partial t} \right \vert_{t=0}
$$

### Influence Function

Assuming that $T$ is Gateaux differentiable at $P$ then a function $\psi:\mathcal{X} \rightarrow \mathbb{R}$ which satisfies $T'(Q-P;P) = \int \psi(x; P)dQ(x)$ is known as the influence function:

$$ \psi(x, P) = T'(\delta_x - P, P) =\left. \frac{\partial T((1-t)P+t\delta_x)}{\partial t} \right \vert_{t=0}$$

### Von Mises

Following a generalization of the Taylor expansion to functionals, the true target quantity $T(Q)$ which we wish to estimate can be expressed as:

$$
T(Q) = T(P) + T'(Q-P;P) + R_2 = T(P) + \int \psi(x;P)dQ(x) + R_2
$$

In words, the true quantity is equal to the estimatable quantity plus the integral of the influence function and some higher order error term(s).

Following a little substitution, the expression can be written as:

$$
T(q) = T(p) + \phi ' \left( \int \nu(p)\right) \int (q-p)\nu ' (p) + R_2
$$

expanding the second term...

$$
T(q) = T(p) + \phi ' \left( \int \nu(p)\right) \left(  \int q\nu ' (p) - \int p \nu ' (p)  \right)+ R_2
$$


### Estimating $T(q)$

As we do not have access to $Q$ we can approximate it using samples from our dataset. This is where our data splitting will come in handy.

The rest of the process is described in line with the code below.


###  Make some imports and define some constants

In [1]:

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.autograd.functional as func
import torch.autograd as grad
from sklearn.neighbors import KernelDensity
torch.pi = torch.tensor(torch.acos(torch.zeros(1)).item() * 2)

# Set up an example for finding the Shannon Entropy of a Gaussian
n = 10000
true_mu = 0
true_sigma = 1
dx = 0.01  # as we are estimating densities with sums we will multiply by dx

### Create the data and the two data splits

In [2]:
x = ((torch.randn(n) + true_mu) * true_sigma).reshape(-1,1)

# data splits
x1 = x[:len(x)//2]
x2 = x[len(x)//2:]

### Derive KDEs for the two data splits

Note that after fitting the kde functions, we also create a 'domain' for x 'r' and use it generate the density 'p_r' by taking the exponential of the sample scores for this domain.

In [3]:
# estimate density using first half of data
kde_ds1 = KernelDensity(kernel='gaussian', bandwidth=0.2).fit(x1)

# estimate density using second half of data
kde_ds2 = KernelDensity(kernel='gaussian', bandwidth=0.2).fit(x2)

# define domain x
r_ds1 = np.arange(-10, 10, dx).reshape(-1,1)
# get density of domain
p_r_ds1 = np.exp(kde_ds1.score_samples(r_ds1)) * dx
print('check estimated density sums to 1: ', p_r_ds1.sum())
p_r_ds1 = torch.tensor(p_r_ds1)

# define domain x
r_ds2 = np.arange(-10, 10, dx).reshape(-1,1)
# get density of domain
p_r_ds2 = np.exp(kde_ds1.score_samples(r_ds2)) * dx
print('check estimated density sums to 1: ', p_r_ds2.sum())
p_r_ds2 = torch.tensor(p_r_ds2)

check estimated density sums to 1:  0.9999999999998879
check estimated density sums to 1:  0.9999999999998879


### Define our functions
Here we define $\nu$, $\phi$ and their combination $T$ which in this example is the Shannon entropy.

In [4]:
def nu(a, dx):
    return - a * torch.log(a/dx) 

def phi(b):
    return b

def entropy(p, dx):
    return phi((nu(p, dx)).sum())

### Calculate the true entropy and the initial estimate
The 'true entropy' is our ground truth for the example, and we calculate it using the analytically derived expression for the entropy of a Gaussian distribution.

The initial estimates are greated using the data split density estimates derived above, and then averaged to provide 'est' which is the initial 'naive' estimate.

In [15]:
# calculate true entropy
true = 0.5 * torch.log(2 * torch.pi * torch.exp(torch.tensor([1])) * true_sigma**2)
print('True value: ', true.item())

# calculate estimated entropy
est_ds1 = entropy(p_r_ds1, dx)
est_ds2 = entropy(p_r_ds2, dx)
est = (est_ds1 + est_ds2) / 2
print('Estimate: ', est.item())
print('Relative estimation error:', (torch.abs(est - true)/true * 100).item(), '%')

True value:  1.4189385175704956
Estimate:  1.4338736315801988
Relative estimation error: 1.0525569915771484 %


### Calculating the Integral of the Influence Function

As above, the integral of the IF is defined as 

$$
\phi ' \left(\int(\nu(p)\right) \left( \int(q \nu'(p)) - \int(p\nu'(p))\right)
$$

We write this as $A(B-C)$ and compute A, B, and C one at a time.

The process is repeated twice, once for the first datasplit, using the second data split in place of $Q$ where needed, and then we repeat this. The second time we swap the splits around, using the first data split in place of $Q$. If this sounds confusing, I have written further details below.

### Part 1A
1. Here we begin by computing $\int \nu(p)$ using the density derived from the first datasplit.

2. We then specify that the resulting estimate requires grad, so that it appears in the computational graph when taking the chain rule using pytorch's autograd library.

3. When then run the estimate for $\int \nu(p)$ through our function $\phi(.)$ (which in our case is the identity function but which has been included here for completeness and generality.

4. Then we compute the gradient of the output w.r.t. the input

5. The pytorch .grad method gives us the resulting value for $\phi'(\int \nu (p) = A$

In [6]:
# A
int_nu_p_ds1 = nu(p_r_ds1, dx).sum()  # 1.
int_nu_p_ds1.requires_grad_(True)   # 2.
phi_int_nu_p_ds1 = phi(int_nu_p_ds1)  # 3.
phi_int_nu_p_ds1.backward(torch.ones(phi_int_nu_p_ds1.shape))  # 4.
A_ds1 = int_nu_p_ds1.grad.data  # 5.

### Part 1B
1. We need to compute $\int q \nu'(p)$. This is in the form of an expectation which we approximate using samples from the *other* datasplit. i.e. we use the kde density derived on the *first* datasplit, to compute sample probabilities for samples from the *second* datasplit. This gives us an approximation for $q$. A similar process is, for example, used in the computation of cross-entropy https://en.wikipedia.org/wiki/Cross_entropy#Estimation

The first step in doing so is therefore to compute the probabilities of these samples. This is what happens in step 1. below.

2. As before, specify that the resulting quantity from step 1. requires a gradient.

3. Now we can compute the expectation for $\nu()$ of the probabilities of these samples

4. ... and compute the gradients through the graph fro $\nu$. Note that we still haven't done anything with $\nu'$, we need $\nu$ first in order to then compute the gradient of the function evaluated at the respective sample probabilities

5. Now we actulaly have $\nu'(.)$ evaluated at the sample probabilities.

6. We take an empirical average of this quantity to get B.

In [7]:
p_xi_ds2 = torch.tensor(np.exp(kde_ds1.score_samples(x2))) * dx  # 1.
p_xi_ds2.requires_grad_(True)  # 2. 
nu_p_xi_ds2 = nu(p_xi_ds2, dx)  # 3.
nu_p_xi_ds2.backward(torch.ones(nu_p_xi_ds2.shape))  # 4.
nu_pr_p_xi_ds2 = p_xi_ds2.grad.data  # 5.
B_ds1 = nu_pr_p_xi_ds2.mean()  # 6.

### Part 1C
1. We need to compute $\int p \nu'(p)$. Some of these steps will be the same as above for the quantity $A$. Once again, we start by computing $\nu (p)$ in order to then get the gradient at $p$. We therefore use the KDE estimated using the first data split, and specify that this requires a gradient.

2. Now we compute $\nu$ of this quantity 

3. ... and compute the gradients

4. And derive the value for $\nu'(.)$ evaluated at our density estimate for $p$.

5. Finally we need to multiply the above quantity by our density estimate for $p$ and sum to acquire the integral.

In [8]:
p_r_ds1.requires_grad_(True)  # 1.
nu_p_ds1 = nu(p_r_ds1, dx)  # 2.
nu_p_ds1.backward(torch.ones(nu_p_ds1.shape))  # 3.
nu_pr_p_ds1 = p_r_ds1.grad.data  # 4. 
C_ds1 = (p_r_ds1 * nu_pr_p_ds1).sum()  # 5.

### Part 1 - Conclusion
Finally we put A, B, and C together to get the first data split estimate for the integral of the influence function!

In [9]:
psi_ds1 = A_ds1 * (B_ds1 - C_ds1)

### Part 2

We now repeat the whole process, but swapping instances of data split 1 for data split 2 and vice versa...

In [10]:
# PART 2

# A
int_nu_p_ds2 = nu(p_r_ds2, dx).sum()
int_nu_p_ds2.requires_grad_(True)
phi_int_nu_p_ds2 = phi(int_nu_p_ds2)
phi_int_nu_p_ds2.backward(torch.ones(phi_int_nu_p_ds2.shape))
A_ds2 = int_nu_p_ds2.grad.data

# B -> 1/n * sum ( nu_pr(p(x_i))) using ds 2

p_xi_ds1 = torch.tensor(np.exp(kde_ds2.score_samples(x1))) * dx
p_xi_ds1.requires_grad_(True)
nu_p_xi_ds1 = nu(p_xi_ds1, dx)
nu_p_xi_ds1.backward(torch.ones(nu_p_xi_ds1.shape))
nu_pr_p_xi_ds1 = p_xi_ds1.grad.data
B_ds2 = nu_pr_p_xi_ds1.mean()

# C
p_r_ds2.requires_grad_(True)
nu_p_ds2 = nu(p_r_ds2, dx)
nu_p_ds2.backward(torch.ones(nu_p_ds2.shape))
nu_pr_p_ds2 = p_r_ds2.grad.data
C_ds2 = (p_r_ds2 * nu_pr_p_ds2).sum()

psi_ds2 = A_ds2 * (B_ds2 - C_ds2)

### Part 3 - the final estimate of the integral of the Influence Function 
We can now put our two estimates of the influence function together to get the average over the full dataset. This is simply $\hat \psi_{DS} = 0.5(\hat \psi_{DS}^1 + \hat \psi_{DS}^2)$

In [11]:
psi = (psi_ds1 + psi_ds2) / 2

### Updating our original Estimate
Following the Von Mises expansion, our updated esimated is simply our old estimate plus the integral of the influence function. We should see that our relative estimation error has decreased...

In [14]:
updated_est = est + psi
print('Updated estimate :', updated_est.item())
print('Relative (naive) estimation error:', (torch.abs(est - true)/true * 100).item(), '%')
print('Relative (updated) estimation error:', (torch.abs(updated_est - true)/true * 100).item(), '%')

Updated estimate : 1.412238771407718
Relative (naive) estimation error: 1.0525569915771484 %
Relative (updated) estimation error: 0.4721698760986328 %
